<a href="https://colab.research.google.com/github/FrodoBaggins87/Neural_Networks/blob/main/FoodDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setting up

In [ ]:
#we need Torch 1.12 + and Torchvision 0.13 + for this study
try:
  import torch, torchvision
  assert int(torch.__version__.split(".")[1])>=12, "Torch version should be 1.12 or above"
  assert int(torchvision.__version__.split(".")[1])>=13, "Torch version should be 0.12 or above"
  print(f"Torch version:{torch.__version__}")
  print(f"torchvision version:{torchvision.__version__}")
except:
  print("Available libraries not updated, downloading updated libraries")
  !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
  import torch, torchvision
  print(f"Torch version:{torch.__version__}")
  print(f"torchvision version:{torchvision.__version__}")

Available libraries not updated, downloading updated libraries
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
Torch version:2.3.1+cu121
torchvision version:0.18.1+cu121


In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
from torch import nn
from torchvision import transforms
try:
  from torchinfo import summary
except:
  !pip install -q torchinfo
  from torchinfo import summary

#Reusable scripts

data_setup.py

In [ ]:
%%writefile data_setup.py
import os
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

NUM_WORKERS= os.cpu_count()
def create_dataloaders(
    train_dir: str,
    test_dir:str,
    train_transform: transforms.Compose,
    test_transform: transforms.Compose,
    batch_size:int,
    num_workers: int=NUM_WORKERS
):

  training_data=datasets.ImageFolder(root=train_dir, transform=train_transform)
  testing_data=datasets.ImageFolder(root=test_dir, transform=test_transform)
  class_names=training_data.classes
  train_dataloader=DataLoader(dataset=training_data,
                              batch_size=batch_size,#sample per dataloader
                              num_workers=num_workers,
                              shuffle=True,
                              pin_memory= True)
  test_dataloader=DataLoader(dataset=testing_data,
                            batch_size=batch_size,
                            num_workers=num_workers,
                            shuffle=False,
                            pin_memory= True)
  return train_dataloader, test_dataloader, class_names


Overwriting data_setup.py


engine.py

In [ ]:
%%writefile engine.py
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple
def train_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer,
              device: torch.device) -> Tuple[float, float]:
  #putting in training mode
  model.train()
  #setup training loss and training accuracy
  train_loss,train_acc=0,0

  for batch,(x,y) in enumerate(dataloader):
    #send data to target device
    x,y=x.to(device),y.to(device)
    #forward pass
    y_pred=model(x)
    #calculate and accumulate losses
    loss=loss_fn(y_pred,y)
    train_loss+=loss.item()
    #optimizer zero grad
    optimizer.zero_grad()
    #loss backward
    loss.backward()
    #optimizer step
    optimizer.step()

    #calculate and accumulate accuracy metric for all batches
    y_pred_class=torch.argmax(torch.softmax(y_pred,dim=1),dim=1)
    train_acc+=(y_pred_class==y).sum().item()/len(y_pred)

  #getting average loss and accuracy for each batch
  train_loss/=len(dataloader)
  train_acc/=len(dataloader)
  return train_loss, train_acc

def test_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
              device: torch.device) -> Tuple[float,float]:
  #putting in eval mode
  model.eval()
  #setup test loss and test accuracy
  test_loss,test_acc=0,0
  #turn on inference context manager
  with torch.inference_mode():
    #loop through dataloader batches
    for batch,(x,y) in enumerate(dataloader):
      #send data to target device
      x,y=x.to(device),y.to(device)
      #forward pass
      test_pred_logits=model(x)
      #calculate and accumulate loss
      loss=loss_fn(test_pred_logits,y)
      test_loss+=loss.item()
      #calculate and accumulate accuracy
      test_pred_labels=torch.argmax(torch.softmax(test_pred_logits,dim=1),dim=1)
      test_acc+=(test_pred_labels==y).sum().item()/len(test_pred_labels)#can probably also use len(test_pred), not sure both should work i think
  #getting average loss and accuracy for each batch
  test_acc/=len(dataloader)
  test_loss/=len(dataloader)
  return test_loss, test_acc

#defining functions and various required parameters
def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          epochs: int,
        device: torch.device) -> Dict[str, list]:
  #create empty results dictionary
  results={"train_loss":[],
           "test_loss":[],
           "train_acc":[],
           "test_acc":[]}
  #looping through train_step() and test_step()
  for epoch in tqdm(range(epochs)):
    train_loss,train_acc=train_step(model=model,
                                    dataloader=train_dataloader,
                                    loss_fn=loss_fn,
                                    optimizer=optimizer,
                                    device=device)
    test_loss, test_acc=test_step(model=model,
                                  dataloader=test_dataloader,
                                  loss_fn=loss_fn,
                                  device=device)
  #print whats happening
    print(
        f"Epoch:{epoch+1}|"
        f"Train Loss:{train_loss:.4f}|"
        f"Training Accuracy: {train_acc:.4f}|"
        f"Test Loss: {test_loss:.4f}|"
        f"Test Accuracy: {test_acc:.4f}"
    )
    #updating result dictionary
    results["train_loss"].append(train_loss)
    results["test_loss"].append(test_loss)
    results["train_acc"].append(train_acc)
    results["test_acc"].append(test_acc)
  return results


Overwriting engine.py


utils.py

In [ ]:
%%writefile utils.py
import torch
from pathlib import Path
def save_model(model:torch.nn.Module,
               target_dir: str,
               model_name:str):
  #creating target directory
  target_dir_path=Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)
  #creating model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path/model_name

  #save the model state_dict
  print(f"Saving model to:{model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Overwriting utils.py


effnet_b2.py

In [ ]:
%%writefile effnet_b2.py
import torch
import torchvision
from torch import nn
def create_effnet_b2(device:str,num_classes:int,seed:int=69):
  #1
  weights=torchvision.models.EfficientNet_B2_Weights.DEFAULT
  transforms=weights.transforms()
  model=torchvision.models.efficientnet_b2(weights=weights).to(device)

  #2. freeze all parameters in all layers
  for param in model.parameters():
    param.requires_grad=False
  #3. set random seeds
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)

  #4. changing classifier layer
  model.classifier= torch.nn.Sequential(nn.Dropout(p=0.2, inplace=True),
                                        nn.Linear(in_features=1408,
                                                  out_features=num_classes,
                                                  bias=True).to(device))
  #5. give name
  model.name='effnet_b2'
  print(f"Making EfficientNet_B2")

  return model,weights,transforms

Overwriting effnet_b2.py


#Device agnostic code

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

#Random seed function

In [ ]:
#set seeds
def set_seeds(seed:int=69):
  "set seed whenever required before torch operations. Default seed = 69"
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)

#Getting an EffNet_B2 model

In [ ]:
import effnet_b2
effnet_model,_,effnet_transforms=effnet_b2.create_effnet_b2(num_classes=101,device=device)


Making EfficientNet_B2


In [ ]:
from torchinfo import summary
summary(effnet_model,
        input_size=(1,3,224,224),
        col_names=['input_size', 'output_size','num_params','trainable'])


Layer (type:depth-idx)                                  Input Shape               Output Shape              Param #                   Trainable
EfficientNet                                            [1, 3, 224, 224]          [1, 101]                  --                        Partial
├─Sequential: 1-1                                       [1, 3, 224, 224]          [1, 1408, 7, 7]           --                        False
│    └─Conv2dNormActivation: 2-1                        [1, 3, 224, 224]          [1, 32, 112, 112]         --                        False
│    │    └─Conv2d: 3-1                                 [1, 3, 224, 224]          [1, 32, 112, 112]         (864)                     False
│    │    └─BatchNorm2d: 3-2                            [1, 32, 112, 112]         [1, 32, 112, 112]         (64)                      False
│    │    └─SiLU: 3-3                                   [1, 32, 112, 112]         [1, 32, 112, 112]         --                        --
│    └─Sequential

#Make separate transform for training dataset

In [ ]:
from torchvision import transforms
food_101_transform=transforms.Compose([transforms.TrivialAugmentWide(),
                                       effnet_transforms])

In [ ]:
print(f'Training data transform:{food_101_transform}')
print(f'Testing data transform:{effnet_transforms}')

Training data transform:Compose(
    TrivialAugmentWide(num_magnitude_bins=31, interpolation=InterpolationMode.NEAREST, fill=None)
    ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)
)
Testing data transform:ImageClassification(
    crop_size=[288]
    resize_size=[288]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


#Getting data

In [ ]:
#make data directory
from pathlib import Path
data_dir=Path('data')

#Getting training data from Food101 dataset
train_data=torchvision.datasets.Food101(root=data_dir,
                                    split='train',
                                    transform=food_101_transform,
                                    download=True)

#Get test data
test_data=torchvision.datasets.Food101(root=data_dir,
                                    split='test',
                                    transform=effnet_transforms,
                                    download=True)

100%|██████████| 4996278331/4996278331 [03:57<00:00, 21077515.23it/s]


Extracting data/food-101.tar.gz to data


In [ ]:
print(train_data.classes)
print(f'Number of training samples: {len(train_data)}')
print(f'Number of testing samples:{len(test_data)}')

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou

Make a subset of the dataset for faster expermenting

In [ ]:
#make a function for splitting a dataset
def split_dataset(dataset:torchvision.datasets, split_size:float=0.2, seed:int=69):
  '''
  Splits a given dataset to get a subset of the training dataset for faster expermentation
  Put dataset in argument dataset and split_size in argument split_size
  '''
  length_1=int(len(dataset)*split_size)
  length_2=len(dataset)-length_1

  print(f"Original dataset length:{len(dataset)}")
  print(f"Length of first split:{length_1}")
  print(f"Length of second split:{length_2}")

  #using torch.utils.data.random_split
  random_split_1,random_split_2=torch.utils.data.random_split(dataset=dataset,
                                                     lengths=[length_1,length_2],
                                                     generator=torch.manual_seed(seed))
  return random_split_1,random_split_2

In [ ]:
#20% of training dataset
train_data_20,_=split_dataset(dataset=train_data,
                                         split_size=0.2,
                                         seed=69)

#20% of testing dataset
test_data_20,_=split_dataset(dataset=test_data,
                                         split_size=0.2,
                                         seed=69)

len(train_data_20),len(test_data_20)

Original dataset length:75750
Length of first split:15150
Length of second split:60600
Original dataset length:25250
Length of first split:5050
Length of second split:20200


(15150, 5050)

In [ ]:
train_data_20

#Make dataloaders for train_data_20 and test_data_20

In [ ]:
import os
import torch

BATCH_SIZE=32
NUM_WORKERS=os.cpu_count()

#Creating training dataloader using torch.utils.data.DataLoader()
train_dataloader_20=torch.utils.data.DataLoader(dataset=train_data_20,
                                                batch_size=32,
                                                shuffle=True,
                                                num_workers=NUM_WORKERS)

#Creating testing dataloader using torch.utils.data.DataLoader()
test_dataloader_20=torch.utils.data.DataLoader(dataset=test_data_20,
                                               batch_size=32,
                                               shuffle=False,
                                               num_workers=NUM_WORKERS)

#making full class_names list
class_names=train_data.classes

#Training classifier layer of eff_net_b2

In [ ]:
#Number of Trainable parameters=142,309
import engine

#set optimizer
optim=torch.optim.Adam(params=effnet_model.parameters(),
                    lr=0.001)
#setup loss function
loss_fn=torch.nn.CrossEntropyLoss()

#train
effnet_results=engine.train(model=effnet_model,
                            train_dataloader=train_dataloader_20,
                            test_dataloader=test_dataloader_20,
                            optimizer=optimizer,
                            loss_fn=loss_fn,
                            epochs=5,
                            device=device)


#Plot loss curves

In [ ]:
#define function to take the values in th e dictionary and plot
def plot_loss_curves(results):
  train_loss=results['train_loss']
  test_loss=results['test_loss']
  train_accuracy=results['train_acc']
  test_accuracy= results['test_acc']
  epochs=range(len(results['train_loss']))
  plt.figure(figsize=(16,8))

  #plotting loss
  plt.subplot(1,2,1)
  plt.plot(epochs, train_loss, label='train loss')
  plt.plot(epochs,test_loss, label='test loss')
  plt.title('Loss')
  plt.xlabel('epochs')
  plt.legend()
  #plotting accuracy
  plt.subplot(1,2,2)
  plt.plot(epochs, train_accuracy, label='train accuracy')
  plt.plot(epochs,test_accuracy, label='test accuracy')
  plt.title('Accuracy')
  plt.xlabel('epochs')
  plt.legend()

In [ ]:
plot_loss_curves(effnet_results)

#Saving the model

In [ ]:
#checking model size
from pathlib import Path
eff_net_b2_size=Path('models/pretrained_eff_net_b2_feature_extractor.pth').stat().st_size//(1024*1024)#division converts bytes to MBs
print(f"The Pretrained EfficientNet_B2 feature extractor has size: {eff_net_b2_size} MB")

In [ ]:
#counting number of parameters
num_parameters=sum(p.numel() for p in effnet_model.parameters())
print(f"The pretrained EfficientNet_B2 feature extractor has {num_parameters} parameters")

In [ ]:
#saving
import utils
utils.save_model(model=effnet_model,
                 target_dir='models',
                 model_name='pretrained_eff_net_feature_extractor.pth')

#Deployment